<a href="https://colab.research.google.com/github/patrickphatnguyen/CIFAR10-Image-Recognition/blob/master/Experiment_4_CIFAR10_LeNet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.0.0-alpha0'

In [2]:
!pip uninstall tensorflow
!pip uninstall tensorflow-gpu
!pip install tensorflow-gpu==2.0.0-alpha0

Uninstalling tensorflow-1.13.1:
  Would remove:
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.13.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.13.1
     |████████████████████████████████| 332.1MB 53kB/s 
     |████████████████████████████████| 419kB 49.7MB/s 
     |████████████████████████████████| 3.0MB 37.0MB/s 
     |████████████████████████████████| 61kB 25.9MB/s 


# Import Library

In [0]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard,ModelCheckpoint


# Build model and load data


In [0]:
def build_model():
    model = Sequential()
    model.add(Conv2D(6, (5, 5), padding='valid', activation = 'relu', kernel_initializer='he_normal', input_shape=(32,32,3)))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(16, (5, 5), padding='valid', activation = 'relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(120, activation = 'relu', kernel_initializer='he_normal'))
    model.add(Dense(84, activation = 'relu', kernel_initializer='he_normal'))
    model.add(Dense(10, activation = 'softmax', kernel_initializer='he_normal'))
    sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0

In [21]:



# Learning rate decay
def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 150:
        return 0.005
    return 0.001


# build network
model = build_model()
print(model.summary())

# set callback
tb_cb = TensorBoard(log_dir='./lenet', histogram_freq=0)
checkpoint_cb=ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True,save_weights_only=False)
lr_decay = LearningRateScheduler(scheduler)
mycb = [lr_decay,tb_cb,checkpoint_cb]

# start train
model.fit(x_train, y_train,
          batch_size=128,
          epochs=35,
          callbacks=mycb,
          validation_data=(x_test, y_test),
          shuffle=True)

# save model
#model.save('lenet.h5')


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 6)         456       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 120)               48120     
_________________________________________________________________
dense_19 (Dense)             (None, 84)               

In [0]:
model.load_weights("best_model.h5")

In [25]:
model.evaluate(x_test,y_test)

10000/10000 [==============================] - 1s 115us/sample - loss: 1.2801 - accuracy: 0.6052


[1.2800772115707397, 0.6052]